### Настройка
Прежде всего, необходимо установить модуль Python `span_marker`. Если необходимо использовать [Weights and Biases](https://wandb.ai/) для логирования, мы можем установить `span_marker` с помощью опции `[wandb]`.

In [1]:
# %pip install span_marker
# %pip install span_marker[wandb]

### Загрузка набора данных
Для этого примера мы загрузим часто используемый набор данных [CoNLL2003](https://huggingface.co/datasets/conll2003) из хаба Hugging Face с помощью 🤗 Datasets.

In [2]:
from datasets import load_dataset

dataset_id = "conll2003"
dataset = load_dataset(dataset_id)
dataset

ClientPayloadError: Response payload is not completed: <ContentLengthError: 400, message='Not enough data to satisfy content length header.'>

In [ ]:
labels = dataset["train"].features["ner_tags"].feature.names
labels

SpanMarker принимает любой набор данных, если у него есть колонки `tokens` и `ner_tags`. `ner_tags` могут быть аннотированы с использованием схем разметки IOB, IOB2, BIOES или BILOU, а также обычных меток без схемы. Этот набор данных CoNLL использует распространенную схему разметки IOB или IOB2 с метками PER, ORG, LOC и MISC.

### Инициализация `SpanMarkerModel`
Модель SpanMarker инициализируется через [SpanMarkerModel.from_pretrained](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.modeling.html#span_marker.modeling.SpanMarkerModel.from_pretrained). Этот метод будет знаком тем, кто знает 🤗 Transformers. Он принимает либо путь к локальной модели, либо имя модели в [хабе Hugging Face](https://huggingface.co/models).

Важно, что модель может быть *либо* энкодером, *либо* уже обученной и сохраненной моделью SpanMarker. Так как мы еще ничего не обучали, мы будем использовать энкодер. Чтобы узнать, как загрузить и использовать сохраненную модель SpanMarker, пожалуйста, ознакомьтесь с блокнотом [Загрузка и Инференс](model_loading.ipynb).

Варианты энкодеров включают BERT и RoBERTa, что означает, что все следующие варианты подходят:

* [prajjwal1/bert-tiny](https://huggingface.co/prajjwal1/bert-tiny)
* [prajjwal1/bert-mini](https://huggingface.co/prajjwal1/bert-mini)
* [prajjwal1/bert-small](https://huggingface.co/prajjwal1/bert-small)
* [prajjwal1/bert-medium](https://huggingface.co/prajjwal1/bert-medium)
* [bert-base-cased](https://huggingface.co/bert-base-cased)
* [bert-large-cased](https://huggingface.co/bert-large-cased)
* [bert-base-multilingual-cased](https://huggingface.co/bert-base-multilingual-cased)
* [bert-base-multilingual-uncased](https://huggingface.co/bert-base-multilingual-uncased)
* [roberta-base](https://huggingface.co/roberta-base)
* [roberta-large](https://huggingface.co/roberta-large)
* [xlm-roberta-base](https://huggingface.co/xlm-roberta-base)
* [xlm-roberta-large](https://huggingface.co/xlm-roberta-large)


Однако не все энкодеры работают: они обязаны принимать `position_ids` в качестве входного аргумента, что исключает DistilBERT, T5, DistilRoBERTa, ALBERT и BART. 

Кроме того, важно учитывать, что модели с учетом регистра (cased) обычно требуют согласованного использования заглавных букв в данных для инференса, соответствующего формату обучающих данных. Проще говоря, если ваши обучающие данные постоянно используют правильный регистр, а данные для инференса — нет, это может привести к неоптимальной производительности. В таких случаях модель без учета регистра (uncased) может оказаться более подходящей. Хотя она может показывать немного более низкие оценки F1 на тестовом наборе, она остается работоспособной независимо от регистра, что делает ее потенциально более эффективной в реальных сценариях.

In [ ]:
from span_marker import SpanMarkerModel, SpanMarkerModelCardData

encoder_id = "roberta-base"
model = SpanMarkerModel.from_pretrained(
    # Required arguments
    encoder_id,
    labels=labels,
    # Optional arguments
    model_max_length=256,
    entity_max_length=6,
    # To improve the generated model card
    model_card_data=SpanMarkerModelCardData(
        language=["en"],
        license="apache-2.0",
        encoder_id=encoder_id,
        dataset_id=dataset_id,
    )
)

Для нас эти предупреждения ожидаемы, так как мы инициализируем `RobertaModel` для новой задачи.

Обратите внимание, что мы передали `SpanMarkerModel.from_pretrained` список наших меток. Это требуется при обучении новой модели с использованием энкодера. Кроме того, мы можем указать некоторые полезные параметры конфигурации из `SpanMarkerConfig`, такие как:

* `model_max_length`: Максимальное количество токенов, которое будет обрабатывать модель. Если вы используете только короткие предложения для своей модели, уменьшение этого числа может ускорить обучение и инференс без потери производительности. По умолчанию используется максимум энкодера или 512, если у энкодера нет максимума.
* `entity_max_length`: Общее количество слов, из которых может состоять одна сущность. По умолчанию 8.
* `model_card_data`: Экземпляр [SpanMarkerModelCardData](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.model_card.html#span_marker.model_card.SpanMarkerModelCardData), в котором вы можете предоставить много полезных данных о вашей модели. Эти данные будут автоматически включены в сгенерированную карточку модели всякий раз, когда модель сохраняется или отправляется в хаб Hugging Face.

### Обучение
На этом этапе наша модель уже готова к обучению! Мы можем импортировать [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) напрямую из 🤗 Transformers, а также `Trainer` из SpanMarker. `Trainer` является подклассом [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) из 🤗 Transformers, который упрощает некоторые задачи, но в остальном работает так же, как обычный `Trainer`

Следующий фрагмент кода показывает некоторые значения по умолчанию. Можно изменять размер пакета (batch size) на меньшее значение, если сработает исключение из-за нехватки памяти.

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="../models/span-marker-roberta-base-conll03",
    report_to="wandb",
    learning_rate=1e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    push_to_hub=False,
    logging_steps=50,
    fp16=True,
    warmup_ratio=0.1,
)

Теперь мы можем создать [Trainer](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.trainer.html#span_marker.trainer.Trainer) из SpanMarker так же, как вы инициализировали бы `Trainer` из 🤗 Transformers.

In [ ]:
from span_marker import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"].select(range(2000)),
)
trainer.train()

И теперь последний шаг — оценить качество модели.

In [ ]:
metrics = trainer.evaluate()
metrics

Кроме того, мы должны оценить модель на тестовом наборе.

In [ ]:
trainer.evaluate(dataset["test"], metric_key_prefix="test")

Сохраняем нашу модель

In [ ]:
trainer.save_model("../models/span-marker-roberta-base-conll03/checkpoint-final")

Если захотим использовать нашу модель то вызываем функцию from_pretrained и указываем путь

In [ ]:
model = SpanMarkerModel.from_pretrained("../models/span-marker-roberta-base-conll03/checkpoint-final")